In [ ]:
### Hello this note book is used primarily for Data Science Project

## Open A new Shopping Mall in San Jose, California

### Introduction
San Jose is a diverse city with many High Technology Firms and being heart of World Famous
Innovation built on Silicon fabricated System Applications not just Products such as AI application
Robotics Internet of Things and many-others.  On the other hands during weekend of holidays there are 
many shoppers spent their family life on leisure activities, relax, scroll around the shopping malls to 
dine and shop spent time and money in the malls.

### Business problem
The objective of this capstone project is to analyze and select the best locations in the city of San Jose, 
California to open a new shopping mall. Use Data Science methodology and machine learning techniques such as 
clustering, this project intends to provide solutions to answer the business question: In the city of San Jose, 
if a property developer is looking to open a new shopping mall, where would you recommend that they open it?

### Target Audience of this project
This project is catering to property developers and investors looking to open or invest in new shopping malls
in capital city of Silicon i.e. San Jose. This project is timely as the city and U.S.A. is currently suffering 
from oversupply of shopping malls due to unprecedent Coronavirus and malls are subjected to closing down due to 
the virus infections. However, when the Virus is gone there is always need for new business investment since 
San Jose is such a high business activities region deserved investment such as Malls or Business Offices.


### Data
To solve the problem we need the following information:

* Use of Neighborhood of San Jose California
* Lattitude and longitude of those neighborhoods which is required to plot the map and use to get the venue data
* Venue Data particularly related to shopping malls and it is used to perform clustering of the neighborhood.

### Source of the data and the method to extract them
The Wiki page (https://en.wikipedia.org/wiki/San_Jose,_California) contains the neighborhood of San Jose 
We will use web scraping with the helps of Python Requests and BeautiSoap packages. Then we can use Python Geocoder 
to obtain the Lattitudes and Longitudes of the Neighborhood. 
We will get the Venue Data using FourSquare API for the Neighborhood. FourSquare provide many catagories of venue data
we are partcularly keen on the one related to Shopping Mall one to help us solve our problem putting forward.

This is a project using many data science using many data science skills from web scraping(Wiki) working with FourSquare API,
data cleaning, data wrangling to machine learning(K-means Clustering) nad map visulaization(Folium). In next section we will
present Methodology where we have taken and dat analysis we have used with machine learning namely K-mean clustering.

In [224]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files
 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import sys
!{sys.executable} -m pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import sys
!{sys.executable} -m pip install folium

import folium # map rendering library

print("Libraries imported.")



Libraries imported.


In [225]:
## Scrap data
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_San_Jose,_California").text

In [226]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')
 

In [227]:
# create a list to store neighborhood data
neighborhoodList = []

In [228]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [229]:
# create a new DataFrame from the list
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,"The Alameda, San Jose"
1,"Almaden Valley, San Jose"
2,"Alum Rock, San Jose"
3,"Alviso, San Jose"
4,"Berryessa, San Jose"


In [230]:
# print the number of rows of the dataframe
kl_df.shape

(47, 1)

In [231]:
## get the geographic coordinates
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, San Jose, California'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [238]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [233]:
coords

[[37.3365261224967, -121.91799838780877],
 [37.23111706290336, -121.89403908790136],
 [37.3581667373175, -121.84408052536502],
 [37.33939000000004, -121.89495999999997],
 [37.381949146988084, -121.86519369729315],
 [37.23916400596684, -121.93754597491845],
 [37.3943672308591, -121.90407725762714],
 [37.32530000000003, -121.92936999999995],
 [37.33939000000004, -121.89495999999997],
 [37.33865000000003, -121.88541999999995],
 [37.281809532981306, -121.85093131398604],
 [37.33939000000004, -121.89495999999997],
 [37.33633000000003, -121.88772999999998],
 [37.33343000000008, -121.89050999999995],
 [37.23687697476185, -121.85424250616896],
 [37.35506066609717, -121.86777251528994],
 [37.29891832269938, -121.86434469721138],
 [37.30597892361225, -121.78180315277551],
 [37.348010000000045, -121.89642999999995],
 [37.340520000000026, -121.84648999999996],
 [37.35176000000007, -121.85964999999999],
 [37.36086000000006, -121.88969999999995],
 [37.31760000000003, -121.81835999999998],
 [37.27698

In [234]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [235]:
# merge the coordinates into the original dataframe
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']

In [236]:
# check the neighborhoods and the coordinates
print(kl_df.shape)
kl_df

(47, 3)


,Neighborhood,Latitude,Longitude
0,"The Alameda, San Jose",37.336526,-121.917998
1,"Almaden Valley, San Jose",37.231117,-121.894039
2,"Alum Rock, San Jose",37.358167,-121.844081
3,"Alviso, San Jose",37.339390,-121.894960
4,"Berryessa, San Jose",37.381949,-121.865194
5,"Blossom Valley, San Jose",37.239164,-121.937546
6,"Buena Vista, San Jose",37.394367,-121.904077
7,"Burbank, Santa Clara County, California",37.325300,-121.929370
8,"Cambrian Park, California",37.339390,-121.894960
9,Chinatowns in San Jose,37.338650,-121.885420


In [239]:
# save the DataFrame as CSV file
kl_df.to_csv("kl_df.csv", index=False)

### 4. Create a map of San Jose with neighborhoods superimposed on top

In [240]:
# get the coordinates of San Jose
address = 'San Jose, California'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of San Jose, California {}, {}.'.format(latitude, longitude))

The geograpical coordinate of San Jose, California 37.3361905, -121.8905833.


In [241]:
# create map of San Jose using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [242]:
# save the map as HTML file
map_kl.save('map_kl.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [243]:
# define Foursquare Credentials and Version
CLIENT_ID = 'JGR1NAYCUDMKVRQGQFLOB1WQ3LQZTEALU4BDSLQXO3JWDLP2' # your Foursquare ID
CLIENT_SECRET = 'KLMQN3HKJBKXW1MOBAMVUON0FCULZX0GS3BAZRL5JYIAKDJ4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' +CLIENT_SECRET)

Your credentails:
CLIENT_ID: JGR1NAYCUDMKVRQGQFLOB1WQ3LQZTEALU4BDSLQXO3JWDLP2
CLIENT_SECRET:KLMQN3HKJBKXW1MOBAMVUON0FCULZX0GS3BAZRL5JYIAKDJ4


### Now, let's get the top 100 venues that are within a radius of 2000 meters.

In [244]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [245]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(4224, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"The Alameda, San Jose",37.336526,-121.917998,Luna Mexican Restaurant,37.333935,-121.915180,Mexican Restaurant
1,"The Alameda, San Jose",37.336526,-121.917998,Wine Affairs,37.333345,-121.914384,Wine Bar
2,"The Alameda, San Jose",37.336526,-121.917998,Zona Rosa,37.333079,-121.914073,Mexican Restaurant
3,"The Alameda, San Jose",37.336526,-121.917998,Central YMCA,37.337796,-121.919896,Gym
4,"The Alameda, San Jose",37.336526,-121.917998,Park Station Hashery,37.333942,-121.924574,New American Restaurant


### Let's check how many venues were returned for each neighorhood

In [246]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Almaden Valley, San Jose",87,87,87,87,87,87
"Alum Rock, San Jose",100,100,100,100,100,100
"Alviso, San Jose",100,100,100,100,100,100
"Berryessa, San Jose",85,85,85,85,85,85
"Blossom Valley, San Jose",33,33,33,33,33,33
"Buena Vista, San Jose",77,77,77,77,77,77
"Burbank, Santa Clara County, California",100,100,100,100,100,100
"Cambrian Park, California",100,100,100,100,100,100
Chinatowns in San Jose,100,100,100,100,100,100


### Let's find out how many unique categories can be curated from all the returned venues

In [247]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 227 uniques categories.


In [248]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Mexican Restaurant', 'Wine Bar', 'Gym', 'New American Restaurant',
       'Grocery Store', 'Sandwich Place', 'Museum', 'Garden', 'Bakery',
       'Coffee Shop', 'Pet Store', 'Pizza Place', 'Playground', 'Buffet',
       'Breakfast Spot', 'Gym / Fitness Center', 'Taco Place',
       'Bubble Tea Shop', 'Clothing Store', 'Theater', 'Smoke Shop',
       'Chinese Restaurant', 'Fast Food Restaurant', 'Hockey Arena',
       'Yoga Studio', 'Frozen Yogurt Shop', 'Juice Bar', 'Trail', 'Café',
       'Thrift / Vintage Store', 'Shipping Store', 'Used Bookstore',
       'Soccer Stadium', 'Burger Joint', 'BBQ Joint', 'Gay Bar',
       'Music Venue', 'Thai Restaurant', 'Sporting Goods Shop',
       'Cajun / Creole Restaurant', 'Nail Salon', 'Ethiopian Restaurant',
       'Park', 'Sushi Restaurant', 'Automotive Shop',
       'Salon / Barbershop', 'Snack Place', 'Italian Restaurant', 'Spa',
       'Health & Beauty Service'], dtype=object)

In [249]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [250]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(4224, 228)


,Neighborhoods,ATM,Accessories Store,African Restaurant,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Big Box Store,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Football Field,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Travel Agency,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"The Alameda, San Jose",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"The Alameda, San Jose",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [251]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(47, 228)


,Neighborhoods,ATM,Accessories Store,African Restaurant,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Big Box Store,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Football Field,Comedy Club,Comic Shop,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Entertainment Service,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Heliport,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotpot Restaurant,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Travel Agency,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,"Almaden Valley, San Jose",0.011494,0.00,0.00,0.000000,0.011494,0.00,0.022989,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.011494,0.000000,0.011494,0.000000,0.000000,0.011494,0.00,0.00,0.00,0.000000,0.00,0.00,0.022989,0.00,0.000000,0.00,0.022989,0.000000,0.000000,0.011494,0.000000,0.000000,0.00,0.00,0.034483,0.00,0.00,0.00,0.000000,0.00,0.034483,0.00,0.00,0.00,0.000000,0.057471,0.000000,0.000000,0.00,0.000000,0.00,0.022989,0.000000,0.00,0.011494,0.00,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.011494,0.022989,0.00,0.000000,0.011494,0.000000,0.00,0.000000,0.000000,0.011494,0.000000,0.000000,0.000000,0.000000,0.011494,0.000000,0.00,0.00,0.011494,0.000

In [252]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

17

In [ ]:
### Create a new DataFrame for Shopping Mall data only

In [253]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]

In [254]:
kl_mall.head(20)

,Neighborhoods,Shopping Mall
0,"Almaden Valley, San Jose",0.022989
1,"Alum Rock, San Jose",0.010000
2,"Alviso, San Jose",0.000000
3,"Berryessa, San Jose",0.000000
4,"Blossom Valley, San Jose",0.000000
5,"Buena Vista, San Jose",0.000000
6,"Burbank, Santa Clara County, California",0.020000
7,"Cambrian Park, California",0.000000
8,Chinatowns in San Jose,0.000000
9,"College Park, San Jose",0.000000


In [ ]:
## 7. Cluster Neighborhoods

### Run k-means to cluster the neighborhoods in San Jose, CA into 3 clusters.

In [255]:
# set number of clusters
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 0, 0, 0, 0, 1, 0, 0, 0], dtype=int32)

In [256]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [257]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,"Almaden Valley, San Jose",0.022989,1
1,"Alum Rock, San Jose",0.010000,2
2,"Alviso, San Jose",0.000000,0
3,"Berryessa, San Jose",0.000000,0
4,"Blossom Valley, San Jose",0.000000,0


In [85]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(47, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Almaden Valley, San Jose",0.00,0,3.147890,101.694050
24,"Naglee Park, San Jose",0.00,0,3.147890,101.694050
25,North San Jose,0.00,0,3.147890,101.694050
26,North San Jose Innovation District,0.00,0,3.147890,101.694050
27,"Palm Haven, San Jose",0.00,0,3.147890,101.694050
28,"Rose Garden, San Jose",0.00,0,3.147890,101.694050
29,"San Jose, California",0.00,0,3.147890,101.694050
31,"Santa Teresa, San Jose",0.00,0,3.147890,101.694050
32,Santana Row,0.00,0,3.192921,101.663964
33,"Seven Trees, San Jose",0.00,0,3.147890,101.694050


## Finally. let's visualize the resulting clusters

In [258]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

KeyError: 'Latitude'

In [259]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

## 8. Examine clusters

### Cluster 0

In [260]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels
2,"Alviso, San Jose",0.0,0
3,"Berryessa, San Jose",0.0,0
4,"Blossom Valley, San Jose",0.0,0
5,"Buena Vista, San Jose",0.0,0
7,"Cambrian Park, California",0.0,0
8,Chinatowns in San Jose,0.0,0
9,"College Park, San Jose",0.0,0
10,"Communications Hill, San Jose",0.0,0
11,"Coyote Valley, California",0.0,0
12,"Downtown Historic District (San Jose, California)",0.0,0


In [264]:
### Cluster 1

kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels
0,"Almaden Valley, San Jose",0.022989,1
6,"Burbank, Santa Clara County, California",0.020000,1
14,"East Foothills, San Jose",0.020000,1
32,Santana Row,0.020000,1
35,Silver Creek Valley,0.040000,1
42,"West San Carlos, San Jose",0.020000,1
46,"Winchester, San Jose",0.020000,1


In [265]:
### Cluster 2

In [266]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels
1,"Alum Rock, San Jose",0.01,2
16,"Edenvale, San Jose",0.01,2
19,King and Story,0.01,2
24,"Naglee Park, San Jose",0.01,2
25,North San Jose,0.01,2
28,"Rose Garden, San Jose",0.01,2
31,"Santa Teresa, San Jose",0.01,2
34,"Shasta Hanchett Park, San Jose",0.01,2
37,South San Jose,0.01,2
39,"St. Leo's, San Jose",0.01,2


## Observation Cluster 2 has most concentrated 10 shopping malls, while cluster 1 has 7 shopping mall while 30 neighborhoods at cluster 0 has no shopping malls, it is recommended developer to invest in cluster 0 virtually without competition.
 

 
 